# Task 2
- To fine tune Mistral and Llama 3.1 models using a dataset that contains text and summaries.

## Approach
- Load the dataset
- Data Preprocessing
- Using Unsloth for faster training and less memory take
- Intializing the training Paramters
- Checking ROUGE score to evaluate performance

In [ ]:
# Installing Essential Libraries

!pip install datasets
!pip install rouge_score
!pip install peft
!pip install -U bitsandbytes transformers accelerate
!pip install xformers==0.0.27

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for ro

In [ ]:
# Importing Essential Libraries

import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

## Loading Dataset from Kaggle

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'newspaper-text-summarization-cnn-dailymail:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1654566%2F2734496%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T111541Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6a846429e40977fb4630b5a715a9e04587ad988e98fd8f7b69218e4d9513d8221f5bf749fb1d15d8ada4aacd5aecc5666e577c6ec950a4b560c97fa1353b5b3003c880f925869caff3219243f86a36e7a2502d614e91cf7f9fdc4793e0f97542e2fa7faa8c7ee234eb5e48d4b7ee145d62be79240b399505800b7dac1394e866c65c42b519a7c41968954bdc19582159ec240f6e9faaaad6fb2d0b68dfbf0ce8a8d123666f2dd5bebc04f651d15e7dedaf2eb21c039fd6871bc5dd1f07d1e4f3ea034956f5fc8a5dad8532b6bfc4ad806725a81d3a5973edb0273c4467c272d6cc8ccd09dd3773ca701bc19cade31764515041a99c582d6c11be39cb37746c23'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 527738644 bytes downloaded
Downloaded and uncompressed: newspaper-text-summarization-cnn-dailymail
Data source import complete.


In [ ]:
train_path = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv"
validation_path = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv"
test_path = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv"

In [ ]:
train_df = pd.read_csv(train_path,engine='python')
validation_df = pd.read_csv(validation_path)
test_df = pd.read_csv(test_path)

In [ ]:
print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

(287113, 3)
(13368, 3)
(11490, 3)


## Data Preprocessing

In [ ]:
# Only 30k samples are taken from training dataset because LLMs perform good in text summarization

train_df = train_df.sample(n=30000, random_state=42)

In [ ]:
# Data Cleaning and Preprocessing
# Remove any non-alphanumeric characters and extra whitespaces
import re
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(' +', ' ', text)
    return text

train_df['article'] = train_df['article'].apply(clean_text)
train_df['highlights'] = train_df['highlights'].apply(clean_text)
validation_df['article'] = validation_df['article'].apply(clean_text)
validation_df['highlights'] = validation_df['highlights'].apply(clean_text)

In [ ]:
# Check for missing data
missing_data = train_df.isnull().sum()
print("Missing Data:")
print(missing_data)
missing_data = validation_df.isnull().sum()
print("Missing Data:")
print(missing_data)

Missing Data:
id            0
article       0
highlights    0
dtype: int64
Missing Data:
id            0
article       0
highlights    0
dtype: int64


In [ ]:
train_df = train_df[['article', 'highlights']]

In [ ]:
val_df = validation_df[['article', 'highlights']]

## Downloading Model, Defining Prompt and Changing dataset according to the prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

# Define the prompt template
prompt_template = """Below is a news article. Write an insightful summary of the news article in 60 words or less.

### Article:
{article}

### Summary:
{highlights}

"""

# Function to format the data
def format_data(example):
    return {
        "text": prompt_template.format(article=example['article'], highlights=example['highlights'])
    }

# Convert DataFrames to Hugging Face Datasets and apply formatting
train_dataset = Dataset.from_pandas(train_df).map(format_data)
val_dataset = Dataset.from_pandas(val_df).map(format_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

### Tokenize the datasets

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

## Initializing Unsloth

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

# Finetuning Mistral-7b


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    eval_dataset=tokenized_val,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.862900
2,2.403100
3,2.184900
4,2.208000
5,2.264300
6,2.060500
7,2.029300
8,1.852700
9,1.973500
10,1.871200


In [ ]:
# Save the model
trainer.save_model("./mistral-7b-finetuned-summarization")

# Finetuning Llama3.1-8B

In [ ]:
model1, tokenizer1 = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model1 = FastLanguageModel.get_peft_model(
    model1,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model1,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    eval_dataset=tokenized_val,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats_llama = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,8.936400
2,9.376500
3,8.908000
4,8.852200
5,9.024800
6,8.688400
7,8.214500
8,7.747600
9,7.816400
10,7.873800


In [ ]:
trainer.save_model("./llama3.1-8b-finetuned-summarization")

### Saving Both the Models

In [ ]:
!zip -r finetuned_model_mistral.zip /content/mistral-7b-finetuned-summarization

  adding: content/mistral-7b-finetuned-summarization/ (stored 0%)
  adding: content/mistral-7b-finetuned-summarization/adapter_config.json (deflated 54%)
  adding: content/mistral-7b-finetuned-summarization/tokenizer.model (deflated 55%)
  adding: content/mistral-7b-finetuned-summarization/adapter_model.safetensors (deflated 8%)
  adding: content/mistral-7b-finetuned-summarization/tokenizer.json (deflated 74%)
  adding: content/mistral-7b-finetuned-summarization/README.md (deflated 66%)
  adding: content/mistral-7b-finetuned-summarization/special_tokens_map.json (deflated 79%)
  adding: content/mistral-7b-finetuned-summarization/training_args.bin (deflated 51%)
  adding: content/mistral-7b-finetuned-summarization/tokenizer_config.json (deflated 68%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp finetuned_model_mistral.zip /content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
!zip -r finetuned_model_llama3.1.zip /content/llama3.1-8b-finetuned-summarization

  adding: content/llama3.1-8b-finetuned-summarization/ (stored 0%)
  adding: content/llama3.1-8b-finetuned-summarization/adapter_config.json (deflated 54%)
  adding: content/llama3.1-8b-finetuned-summarization/tokenizer.model (deflated 55%)
  adding: content/llama3.1-8b-finetuned-summarization/adapter_model.safetensors (deflated 7%)
  adding: content/llama3.1-8b-finetuned-summarization/tokenizer.json (deflated 74%)
  adding: content/llama3.1-8b-finetuned-summarization/README.md (deflated 66%)
  adding: content/llama3.1-8b-finetuned-summarization/special_tokens_map.json (deflated 79%)
  adding: content/llama3.1-8b-finetuned-summarization/training_args.bin (deflated 51%)
  adding: content/llama3.1-8b-finetuned-summarization/tokenizer_config.json (deflated 68%)


In [ ]:
!cp finetuned_model_llama3.1.zip /content/drive/MyDrive/

## Testing the Models


In [ ]:
test_df = pd.read_csv(test_path)

In [ ]:
import re
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(' +', ' ', text)
    return text

test_df['article'] = test_df['article'].apply(clean_text)
test_df['highlights'] = test_df['highlights'].apply(clean_text)

In [ ]:
missing_data = test_df.isnull().sum()
print("Missing Data:")
print(missing_data)

Missing Data:
id            0
article       0
highlights    0
dtype: int64


In [ ]:
test_df = test_df[['article', 'highlights']]

In [ ]:
testing = test_df.sample(n=1, random_state=101)

In [ ]:
from datasets import load_metric
rouge = load_metric('rouge')

<ipython-input-31-e01c26fba65d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/finetuned_model_mistral.zip -d /content/


Archive:  /content/drive/MyDrive/finetuned_model_mistral.zip
   creating: /content/content/mistral-7b-finetuned-summarization/
  inflating: /content/content/mistral-7b-finetuned-summarization/adapter_config.json  
  inflating: /content/content/mistral-7b-finetuned-summarization/tokenizer.model  
  inflating: /content/content/mistral-7b-finetuned-summarization/adapter_model.safetensors  
  inflating: /content/content/mistral-7b-finetuned-summarization/tokenizer.json  
  inflating: /content/content/mistral-7b-finetuned-summarization/README.md  
  inflating: /content/content/mistral-7b-finetuned-summarization/special_tokens_map.json  
  inflating: /content/content/mistral-7b-finetuned-summarization/training_args.bin  
  inflating: /content/content/mistral-7b-finetuned-summarization/tokenizer_config.json  


In [ ]:
!unzip /content/drive/MyDrive/finetuned_model_llama3.1.zip -d /content/

Archive:  /content/drive/MyDrive/finetuned_model_llama3.1.zip
   creating: /content/content/llama3.1-8b-finetuned-summarization/
  inflating: /content/content/llama3.1-8b-finetuned-summarization/adapter_config.json  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/tokenizer.model  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/adapter_model.safetensors  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/tokenizer.json  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/README.md  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/special_tokens_map.json  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/training_args.bin  
  inflating: /content/content/llama3.1-8b-finetuned-summarization/tokenizer_config.json  


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_llama = AutoModelForCausalLM.from_pretrained("/content/content/llama3.1-8b-finetuned-summarization")
tokenizer_llama = AutoTokenizer.from_pretrained("/content/content/llama3.1-8b-finetuned-summarization")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
 generated_summaries_llama= []

In [ ]:
for article in input_test:
    # Tokenize the input text
    input_ids = tokenizer_llama(article, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    # Generate prediction (summary)
    outputs = model_llama.generate(input_ids, max_new_tokens=50, num_beams=5, early_stopping=False)

    # Decode the generated text
    summary = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)

    if article in summary:
        summary = summary.replace(article, "").strip()

    # Append to the list of generated summaries
    generated_summaries_llama.append(summary)

In [ ]:
generated_summaries_llama

['43 treated her three youngest children Princess Isabella seven and twins Princess Josephine and Prince Vincent four to a day out at a farm in Kirke Hyllinge a village in Zealand Making the most of the sunny weather the quartet']

In [ ]:
testing['highlights'][269]

'Crown Princess Mary spent Sunday on the farm with her children \nHusband Crown Prince Frederik was absent as was Prince Christian nine \nSpent the day out petting calves and inspecting a herd of cows \nEvent took place in the tiny village of Kirke Hyllinge in Zealand '

In [ ]:
results_llama = rouge.compute(predictions=generated_summaries_llama, references=reference_summaries)

In [ ]:
results_llama

{'rouge1': AggregateScore(low=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913), mid=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913), high=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913)),
 'rouge2': AggregateScore(low=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144), mid=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144), high=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144)),
 'rougeL': AggregateScore(low=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721), mid=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721), high=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721)),
 'rougeLsum': AggregateScore(low=Score(precision=0.3, recall=0.2608695652173913, fmeasure=0.27906976744186

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_mistral = AutoModelForCausalLM.from_pretrained("/content/content/mistral-7b-finetuned-summarization")
tokenizer_mistral = AutoTokenizer.from_pretrained("/content/content/mistral-7b-finetuned-summarization")

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

In [ ]:
 generated_summaries_mistral= []

In [ ]:
for article in input_test:
    # Tokenize the input text
    input_ids = tokenizer_mistral(article, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    # Generate prediction (summary)
    outputs = model_mistral.generate(input_ids, max_new_tokens=50, num_beams=5, early_stopping=False)

    # Decode the generated text
    summary = tokenizer_mistral.decode(outputs[0], skip_special_tokens=True)

    if article in summary:
        summary = summary.replace(article, "").strip()

    # Append to the list of generated summaries
    generated_summaries_mistral.append(summary)

In [ ]:
generated_summaries_mistral

['### Also read:\n\nDenmarks Crown Princess Mary and her three youngest children Princess Isabella seven and twins Princess Josephine and Prince Vincent four treated to a day out at a farm in Kirke Hyllinge a village in']

In [ ]:
results_mistral = rouge.compute(predictions=generated_summaries_mistral, references=reference_summaries)

In [ ]:
results_mistral

{'rouge1': AggregateScore(low=Score(precision=0.4444444444444444, recall=0.34782608695652173, fmeasure=0.3902439024390244), mid=Score(precision=0.4444444444444444, recall=0.34782608695652173, fmeasure=0.3902439024390244), high=Score(precision=0.4444444444444444, recall=0.34782608695652173, fmeasure=0.3902439024390244)),
 'rouge2': AggregateScore(low=Score(precision=0.11428571428571428, recall=0.08888888888888889, fmeasure=0.1), mid=Score(precision=0.11428571428571428, recall=0.08888888888888889, fmeasure=0.1), high=Score(precision=0.11428571428571428, recall=0.08888888888888889, fmeasure=0.1)),
 'rougeL': AggregateScore(low=Score(precision=0.3611111111111111, recall=0.2826086956521739, fmeasure=0.3170731707317073), mid=Score(precision=0.3611111111111111, recall=0.2826086956521739, fmeasure=0.3170731707317073), high=Score(precision=0.3611111111111111, recall=0.2826086956521739, fmeasure=0.3170731707317073)),
 'rougeLsum': AggregateScore(low=Score(precision=0.3611111111111111, recall=0.2

In [ ]:
results_llama

{'rouge1': AggregateScore(low=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913), mid=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913), high=Score(precision=0.475, recall=0.41304347826086957, fmeasure=0.44186046511627913)),
 'rouge2': AggregateScore(low=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144), mid=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144), high=Score(precision=0.07692307692307693, recall=0.06666666666666667, fmeasure=0.07142857142857144)),
 'rougeL': AggregateScore(low=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721), mid=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721), high=Score(precision=0.275, recall=0.2391304347826087, fmeasure=0.2558139534883721)),
 'rougeLsum': AggregateScore(low=Score(precision=0.3, recall=0.2608695652173913, fmeasure=0.27906976744186

# Conclusion

Llama 3.1 8b is performing better than the Mistral 7b for summarization.

#Reason
The value of Rouge-L and Rouge-2 is higher for LLama3.1 and Rouge-L and Rouge-2 indicate the coherence and structural accuracy of the summary.
